In [1]:
import certifi
import pandas as pd
import json
import urllib3
from urllib3 import request
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
from datetime import date
today = date.today()


In [2]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
state = 'AL'
api_address="https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=incidentType%20eq%20%27Severe%20Storm(s)%27%20and%20state%20eq%20%27"+state+"%27"
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
r = http.request('GET', api_address)
r.status


200

In [3]:
# for the state of Wyoming - WY from 2019-01-01 -> No data is Present
#Change it to Alabama - AL from 2019-01-01 to view data
data = json.loads(r.data.decode('utf-8'))
if data['DisasterDeclarationsSummaries'] == []:
        error = "No Data Avaliable in this Time Period for the state "+state
        print(error)    
else:
    df = pd.json_normalize(data, 'DisasterDeclarationsSummaries')
df['fips'] = df['fipsStateCode']+df['fipsCountyCode']
df['values'] = 1
df['incidentdate'] = df.declarationDate.str[:10]
df.head()
# for date in df['incidentdate']:
#     if today == date:
#         print("Send Message")
#     else:
#         print("No Message")

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,hash,lastRefresh,id,fips,values,incidentdate
0,DR-285-AL,285,AL,DR,1970-04-09T05:00:00.000Z,1970,Severe Storm(s),"HEAVY RAINS, TORNADOES & FLOODING",False,True,...,017,99017,Chambers (County),70012,6bd17396dd3aa727319451684000651b,2020-10-05T14:21:23.497Z,5f7b2be331a8c6681cfb6103,01017,1,1970-04-09
1,DR-285-AL,285,AL,DR,1970-04-09T05:00:00.000Z,1970,Severe Storm(s),"HEAVY RAINS, TORNADOES & FLOODING",False,True,...,073,99073,Jefferson (County),70012,21f251b9aeef581785e2fedc9d7a0493,2020-10-05T14:21:23.499Z,5f7b2be331a8c6681cfb6117,01073,1,1970-04-09
2,DR-488-AL,488,AL,DR,1975-10-02T04:00:00.000Z,1976,Severe Storm(s),"SEVERE STORMS, TORNADOES & FLOODING",False,True,...,011,99011,Bullock (County),76020,4428073788d59d5101643a62c7d7e9f3,2020-10-05T14:21:27.642Z,5f7b2be731a8c6681cfb8f88,01011,1,1975-10-02
3,DR-488-AL,488,AL,DR,1975-10-02T04:00:00.000Z,1976,Severe Storm(s),"SEVERE STORMS, TORNADOES & FLOODING",False,True,...,017,99017,Chambers (County),76020,24692c0eed9fac2e61fc3925538ce01d,2020-10-05T14:21:27.644Z,5f7b2be731a8c6681cfb8f8c,01017,1,1975-10-02
4,DR-488-AL,488,AL,DR,1975-10-02T04:00:00.000Z,1976,Severe Storm(s),"SEVERE STORMS, TORNADOES & FLOODING",False,True,...,031,99031,Coffee (County),76020,9fef756c2da3a7d09fdfb1f863c27123,2020-10-05T14:21:27.646Z,5f7b2be731a8c6681cfb8f96,01031,1,1975-10-02


In [5]:
fig = px.choropleth_mapbox(df, geojson=counties, locations='fips',
                            color_continuous_scale="Viridis",
                            range_color=(0, 12),
                            mapbox_style="open-street-map",
                            zoom=3.5, center = {"lat": 37.0902, "lon": -95.7129},
                            opacity=0.5,
                            hover_name = df['designatedArea'],
                            hover_data=["incidentdate", "declarationTitle"]
                            )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()